# rag-search


## Import packages

In [1]:
import docx
import PyPDF2
from pptx import Presentation
from os import listdir
from os.path import isfile, join, isdir
from langchain_text_splitters import TokenTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from langchain_qdrant import Qdrant

In [2]:
def list_files(dir):
    # Initialize a empty list to store the file path
    arquivo_list = []
    
    # Iterate every files and directories on specif directory
    for f in listdir(dir):
        
        # If it's a file, add to the list
        if isfile(join(dir, f)):
            arquivo_list.append(join(dir, f))
        
        # If it is a directory, call the function recursively and add the results to the list
        elif isdir(join(dir, f)):
            arquivo_list += list_files(join(dir, f))
    
    # Return the list of files
    return arquivo_list